In [18]:
import cv2
import os
import numpy as np

def segmentasi_buah_pisang_hsv(gambar):
    # Konversi gambar ke ruang warna HSV
    hsv = cv2.cvtColor(gambar, cv2.COLOR_BGR2HSV)

    # Definisikan rentang nilai warna kuning untuk segmentasi buah pisang
    lower_yellow = np.array([0, 0, 0])
    upper_yellow = np.array([255, 255, 255])

    # Buat mask berdasarkan rentang nilai warna kuning
    masked = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Menghitung jumlah piksel kuning dalam mask
    yellow_pixels = cv2.countNonZero(masked)
    
    # Mencari kontur dari mask
    contours, _ = cv2.findContours(masked, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    max_area = 0
    max_contour = None
    
    # Mencari kontur kuning dengan luas terbesar
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if area > max_area:
            max_area = area
            max_contour = contour
    
    # Mengekstrak bounding box dari kontur terbesar
    x, y, w, h = cv2.boundingRect(max_contour)
    
    # Menambahkan margin pada bounding rectangle
    margin = 50
    x1 = max(0, x - margin)
    y1 = max(0, y - margin)
    x2 = min(gambar.shape[1], x + w + margin)
    y2 = min(gambar.shape[0], y + h + margin)
    
    # Menghitung koordinat tengah bounding box
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2
    
    # Mengubah posisi kotak berdasarkan koordinat tengah bounding box
    kotak_width = 50
    kotak_height = 70
    kotak_x1 = max(0, center_x - kotak_width // 2)
    kotak_y1 = max(0, center_y - kotak_height // 2)
    kotak_x2 = min(gambar.shape[1], center_x + kotak_width // 2)
    kotak_y2 = min(gambar.shape[0], center_y + kotak_height // 2)
    
    # Membuat kotak dengan ukuran 100x200 di sekitar area dengan kuning paling banyak
    kotak = np.zeros((kotak_height, kotak_width, 3), dtype=np.uint8)
    kotak[:kotak_y2-kotak_y1, :kotak_x2-kotak_x1] = gambar[kotak_y1:kotak_y1+kotak_y2-kotak_y1, kotak_x1:kotak_x1+kotak_x2-kotak_x1]

    return kotak


def segmentasi_buah_pisang_rgb(gambar):
    
    # Konversi gambar ke ruang warna RGB
    image_rgb = cv2.cvtColor(gambar, cv2.COLOR_BGR2RGB)

   # Definisikan rentang nilai warna kuning untuk segmentasi buah pisang
    lower_yellow = np.array([0, 0, 0])
    upper_yellow = np.array([255, 255, 255])

    # Buat mask berdasarkan rentang nilai warna kuning
    masked = cv2.inRange(image_rgb, lower_yellow, upper_yellow)
    
    # Menghitung jumlah piksel kuning dalam mask
    yellow_pixels = cv2.countNonZero(masked)
    
    # Mencari kontur dari mask
    contours, _ = cv2.findContours(masked, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    max_area = 0
    max_contour = None
    
    # Mencari kontur kuning dengan luas terbesar
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if area > max_area:
            max_area = area
            max_contour = contour
    
    # Mengekstrak bounding box dari kontur terbesar
    x, y, w, h = cv2.boundingRect(max_contour)
    
    # Menambahkan margin pada bounding rectangle
    margin = 50
    x1 = max(0, x - margin)
    y1 = max(0, y - margin)
    x2 = min(gambar.shape[1], x + w + margin)
    y2 = min(gambar.shape[0], y + h + margin)
    
    # Menghitung koordinat tengah bounding box
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2
    
    # Mengubah posisi kotak berdasarkan koordinat tengah bounding box
    kotak_width = 50
    kotak_height = 70
    kotak_x1 = max(0, center_x - kotak_width // 2)
    kotak_y1 = max(0, center_y - kotak_height // 2)
    kotak_x2 = min(gambar.shape[1], center_x + kotak_width // 2)
    kotak_y2 = min(gambar.shape[0], center_y + kotak_height // 2)
    
    # Membuat kotak dengan ukuran 100x200 di sekitar area dengan kuning paling banyak
    kotak = np.zeros((kotak_height, kotak_width, 3), dtype=np.uint8)
    kotak[:kotak_y2-kotak_y1, :kotak_x2-kotak_x1] = gambar[kotak_y1:kotak_y1+kotak_y2-kotak_y1, kotak_x1:kotak_x1+kotak_x2-kotak_x1]

    return kotak

def segmentasi_buah_pisang(input_dir, output_dir):
    # Buat direktori output jika belum ada
    os.makedirs(output_dir, exist_ok=True)

    # Dapatkan daftar file dalam direktori input
    file_list = os.listdir(input_dir)

    # Melakukan segmentasi pada setiap file gambar
    for file_name in file_list:
        # Baca gambar
        gambar = cv2.imread(os.path.join(input_dir, file_name))

        # Memeriksa jumlah piksel dengan metode segmentasi HSV
        segmented_hsv = segmentasi_buah_pisang_hsv(gambar)
        jumlah_pixel_hsv = np.sum(segmented_hsv.astype(bool))

        # Memeriksa jumlah piksel dengan metode segmentasi RGB
        segmented_rgb = segmentasi_buah_pisang_rgb(gambar)
        jumlah_pixel_rgb = np.sum(segmented_rgb.astype(bool))
        
        # Memutuskan metode segmentasi yang akan digunakan
        if jumlah_pixel_hsv < 24000:
            segmented_img = segmented_rgb
            jumlah_pixel = jumlah_pixel_rgb
        else:
            segmented_img = segmented_hsv
            jumlah_pixel = jumlah_pixel_hsv

        # Ubah nama file hasil segmentasi
        segmented_file_name = "segmented_" + file_name

        # Simpan hasil segmentasi ke direktori output
        cv2.imwrite(os.path.join(output_dir, segmented_file_name), segmented_img)

        print(f"Segmentasi selesai: {file_name} -> {segmented_file_name}")

# Contoh penggunaan
input_directory = "T:/PCD/Banana_Condensed_Three_Cat/all"
output_directory = "T:/PCD/Banana_Condensed_Three_Cat/segmented"
segmentasi_buah_pisang(input_directory, output_directory)


Segmentasi selesai: overripe (10).jpg -> segmented_overripe (10).jpg
Segmentasi selesai: overripe (100).jpg -> segmented_overripe (100).jpg
Segmentasi selesai: overripe (101).jpg -> segmented_overripe (101).jpg
Segmentasi selesai: overripe (102).jpg -> segmented_overripe (102).jpg
Segmentasi selesai: overripe (103).jpg -> segmented_overripe (103).jpg
Segmentasi selesai: overripe (104).jpg -> segmented_overripe (104).jpg
Segmentasi selesai: overripe (105).jpg -> segmented_overripe (105).jpg
Segmentasi selesai: overripe (106).jpg -> segmented_overripe (106).jpg
Segmentasi selesai: overripe (107).jpg -> segmented_overripe (107).jpg
Segmentasi selesai: overripe (108).jpg -> segmented_overripe (108).jpg
Segmentasi selesai: overripe (109).jpg -> segmented_overripe (109).jpg
Segmentasi selesai: overripe (11).jpg -> segmented_overripe (11).jpg
Segmentasi selesai: overripe (110).jpg -> segmented_overripe (110).jpg
Segmentasi selesai: overripe (111).jpg -> segmented_overripe (111).jpg
Segmentasi

Segmentasi selesai: overripe (21).jpg -> segmented_overripe (21).jpg
Segmentasi selesai: overripe (210).jpg -> segmented_overripe (210).jpg
Segmentasi selesai: overripe (211).jpg -> segmented_overripe (211).jpg
Segmentasi selesai: overripe (212).jpg -> segmented_overripe (212).jpg
Segmentasi selesai: overripe (213).jpg -> segmented_overripe (213).jpg
Segmentasi selesai: overripe (214).jpg -> segmented_overripe (214).jpg
Segmentasi selesai: overripe (215).jpg -> segmented_overripe (215).jpg
Segmentasi selesai: overripe (216).jpg -> segmented_overripe (216).jpg
Segmentasi selesai: overripe (217).jpg -> segmented_overripe (217).jpg
Segmentasi selesai: overripe (218).jpg -> segmented_overripe (218).jpg
Segmentasi selesai: overripe (219).jpg -> segmented_overripe (219).jpg
Segmentasi selesai: overripe (22).jpg -> segmented_overripe (22).jpg
Segmentasi selesai: overripe (220).jpg -> segmented_overripe (220).jpg
Segmentasi selesai: overripe (221).jpg -> segmented_overripe (221).jpg
Segmentasi

Segmentasi selesai: overripe (325).jpg -> segmented_overripe (325).jpg
Segmentasi selesai: overripe (326).jpg -> segmented_overripe (326).jpg
Segmentasi selesai: overripe (327).jpg -> segmented_overripe (327).jpg
Segmentasi selesai: overripe (328).jpg -> segmented_overripe (328).jpg
Segmentasi selesai: overripe (329).jpg -> segmented_overripe (329).jpg
Segmentasi selesai: overripe (33).jpg -> segmented_overripe (33).jpg
Segmentasi selesai: overripe (330).jpg -> segmented_overripe (330).jpg
Segmentasi selesai: overripe (331).jpg -> segmented_overripe (331).jpg
Segmentasi selesai: overripe (332).jpg -> segmented_overripe (332).jpg
Segmentasi selesai: overripe (333).jpg -> segmented_overripe (333).jpg
Segmentasi selesai: overripe (334).jpg -> segmented_overripe (334).jpg
Segmentasi selesai: overripe (335).jpg -> segmented_overripe (335).jpg
Segmentasi selesai: overripe (336).jpg -> segmented_overripe (336).jpg
Segmentasi selesai: overripe (337).jpg -> segmented_overripe (337).jpg
Segmenta

Segmentasi selesai: overripe (5).jpg -> segmented_overripe (5).jpg
Segmentasi selesai: overripe (50).jpg -> segmented_overripe (50).jpg
Segmentasi selesai: overripe (51).jpg -> segmented_overripe (51).jpg
Segmentasi selesai: overripe (52).jpg -> segmented_overripe (52).jpg
Segmentasi selesai: overripe (53).jpg -> segmented_overripe (53).jpg
Segmentasi selesai: overripe (54).jpg -> segmented_overripe (54).jpg
Segmentasi selesai: overripe (55).jpg -> segmented_overripe (55).jpg
Segmentasi selesai: overripe (56).jpg -> segmented_overripe (56).jpg
Segmentasi selesai: overripe (57).jpg -> segmented_overripe (57).jpg
Segmentasi selesai: overripe (58).jpg -> segmented_overripe (58).jpg
Segmentasi selesai: overripe (59).jpg -> segmented_overripe (59).jpg
Segmentasi selesai: overripe (6).jpg -> segmented_overripe (6).jpg
Segmentasi selesai: overripe (60).jpg -> segmented_overripe (60).jpg
Segmentasi selesai: overripe (61).jpg -> segmented_overripe (61).jpg
Segmentasi selesai: overripe (62).jpg 

Segmentasi selesai: ripe (166).jpg -> segmented_ripe (166).jpg
Segmentasi selesai: ripe (167).jpg -> segmented_ripe (167).jpg
Segmentasi selesai: ripe (168).jpg -> segmented_ripe (168).jpg
Segmentasi selesai: ripe (169).jpg -> segmented_ripe (169).jpg
Segmentasi selesai: ripe (17).jpg -> segmented_ripe (17).jpg
Segmentasi selesai: ripe (170).jpg -> segmented_ripe (170).jpg
Segmentasi selesai: ripe (171).jpg -> segmented_ripe (171).jpg
Segmentasi selesai: ripe (172).jpg -> segmented_ripe (172).jpg
Segmentasi selesai: ripe (173).jpg -> segmented_ripe (173).jpg
Segmentasi selesai: ripe (174).jpg -> segmented_ripe (174).jpg
Segmentasi selesai: ripe (175).jpg -> segmented_ripe (175).jpg
Segmentasi selesai: ripe (176).jpg -> segmented_ripe (176).jpg
Segmentasi selesai: ripe (177).jpg -> segmented_ripe (177).jpg
Segmentasi selesai: ripe (178).jpg -> segmented_ripe (178).jpg
Segmentasi selesai: ripe (179).jpg -> segmented_ripe (179).jpg
Segmentasi selesai: ripe (18).jpg -> segmented_ripe (18).

Segmentasi selesai: ripe (290).jpg -> segmented_ripe (290).jpg
Segmentasi selesai: ripe (291).jpg -> segmented_ripe (291).jpg
Segmentasi selesai: ripe (292).jpg -> segmented_ripe (292).jpg
Segmentasi selesai: ripe (293).jpg -> segmented_ripe (293).jpg
Segmentasi selesai: ripe (294).jpg -> segmented_ripe (294).jpg
Segmentasi selesai: ripe (295).jpg -> segmented_ripe (295).jpg
Segmentasi selesai: ripe (296).jpg -> segmented_ripe (296).jpg
Segmentasi selesai: ripe (297).jpg -> segmented_ripe (297).jpg
Segmentasi selesai: ripe (298).jpg -> segmented_ripe (298).jpg
Segmentasi selesai: ripe (299).jpg -> segmented_ripe (299).jpg
Segmentasi selesai: ripe (3).jpg -> segmented_ripe (3).jpg
Segmentasi selesai: ripe (30).jpg -> segmented_ripe (30).jpg
Segmentasi selesai: ripe (300).jpg -> segmented_ripe (300).jpg
Segmentasi selesai: ripe (301).jpg -> segmented_ripe (301).jpg
Segmentasi selesai: ripe (302).jpg -> segmented_ripe (302).jpg
Segmentasi selesai: ripe (303).jpg -> segmented_ripe (303).jp

Segmentasi selesai: underripe (110).jpg -> segmented_underripe (110).jpg
Segmentasi selesai: underripe (111).jpg -> segmented_underripe (111).jpg
Segmentasi selesai: underripe (112).jpg -> segmented_underripe (112).jpg
Segmentasi selesai: underripe (113).jpg -> segmented_underripe (113).jpg
Segmentasi selesai: underripe (114).jpg -> segmented_underripe (114).jpg
Segmentasi selesai: underripe (115).jpg -> segmented_underripe (115).jpg
Segmentasi selesai: underripe (116).jpg -> segmented_underripe (116).jpg
Segmentasi selesai: underripe (117).jpg -> segmented_underripe (117).jpg
Segmentasi selesai: underripe (118).jpg -> segmented_underripe (118).jpg
Segmentasi selesai: underripe (119).jpg -> segmented_underripe (119).jpg
Segmentasi selesai: underripe (12).jpg -> segmented_underripe (12).jpg
Segmentasi selesai: underripe (120).jpg -> segmented_underripe (120).jpg
Segmentasi selesai: underripe (121).jpg -> segmented_underripe (121).jpg
Segmentasi selesai: underripe (122).jpg -> segmented_

Segmentasi selesai: underripe (220).jpg -> segmented_underripe (220).jpg
Segmentasi selesai: underripe (221).jpg -> segmented_underripe (221).jpg
Segmentasi selesai: underripe (222).jpg -> segmented_underripe (222).jpg
Segmentasi selesai: underripe (223).jpg -> segmented_underripe (223).jpg
Segmentasi selesai: underripe (224).jpg -> segmented_underripe (224).jpg
Segmentasi selesai: underripe (225).jpg -> segmented_underripe (225).jpg
Segmentasi selesai: underripe (226).jpg -> segmented_underripe (226).jpg
Segmentasi selesai: underripe (227).jpg -> segmented_underripe (227).jpg
Segmentasi selesai: underripe (228).jpg -> segmented_underripe (228).jpg
Segmentasi selesai: underripe (229).jpg -> segmented_underripe (229).jpg
Segmentasi selesai: underripe (23).jpg -> segmented_underripe (23).jpg
Segmentasi selesai: underripe (230).jpg -> segmented_underripe (230).jpg
Segmentasi selesai: underripe (231).jpg -> segmented_underripe (231).jpg
Segmentasi selesai: underripe (232).jpg -> segmented_

Segmentasi selesai: underripe (335).jpg -> segmented_underripe (335).jpg
Segmentasi selesai: underripe (336).jpg -> segmented_underripe (336).jpg
Segmentasi selesai: underripe (337).jpg -> segmented_underripe (337).jpg
Segmentasi selesai: underripe (338).jpg -> segmented_underripe (338).jpg
Segmentasi selesai: underripe (339).jpg -> segmented_underripe (339).jpg
Segmentasi selesai: underripe (34).jpg -> segmented_underripe (34).jpg
Segmentasi selesai: underripe (340).jpg -> segmented_underripe (340).jpg
Segmentasi selesai: underripe (341).jpg -> segmented_underripe (341).jpg
Segmentasi selesai: underripe (342).jpg -> segmented_underripe (342).jpg
Segmentasi selesai: underripe (343).jpg -> segmented_underripe (343).jpg
Segmentasi selesai: underripe (344).jpg -> segmented_underripe (344).jpg
Segmentasi selesai: underripe (345).jpg -> segmented_underripe (345).jpg
Segmentasi selesai: underripe (346).jpg -> segmented_underripe (346).jpg
Segmentasi selesai: underripe (347).jpg -> segmented_

Segmentasi selesai: underripe (79).jpg -> segmented_underripe (79).jpg
Segmentasi selesai: underripe (8).jpg -> segmented_underripe (8).jpg
Segmentasi selesai: underripe (80).jpg -> segmented_underripe (80).jpg
Segmentasi selesai: underripe (81).jpg -> segmented_underripe (81).jpg
Segmentasi selesai: underripe (82).jpg -> segmented_underripe (82).jpg
Segmentasi selesai: underripe (83).jpg -> segmented_underripe (83).jpg
Segmentasi selesai: underripe (84).jpg -> segmented_underripe (84).jpg
Segmentasi selesai: underripe (85).jpg -> segmented_underripe (85).jpg
Segmentasi selesai: underripe (86).jpg -> segmented_underripe (86).jpg
Segmentasi selesai: underripe (87).jpg -> segmented_underripe (87).jpg
Segmentasi selesai: underripe (88).jpg -> segmented_underripe (88).jpg
Segmentasi selesai: underripe (89).jpg -> segmented_underripe (89).jpg
Segmentasi selesai: underripe (9).jpg -> segmented_underripe (9).jpg
Segmentasi selesai: underripe (90).jpg -> segmented_underripe (90).jpg
Segmentasi